# Create a slice with PTP for precise time measurements

For running MF timestamp or OWL (one-way latency measurement) tools, slice nodes must meet prerequisites: 

+ Git and Dockerhub must be reachable 
+ Docker has to be running
+ PTP (Precision Time Protocol) clock must be running/

This notebook creates a 3-node slice and sets up all the the above

(Tested on 2023/08/18)

## Import the FABlib Library


In [1]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

try:
    fablib = fablib_manager()
                     
    fablib.show_config()
except Exception as e:
    print(f"Exception: {e}")

Credential Manager,cm.fabric-testbed.net
Orchestrator,orchestrator.fabric-testbed.net
Token File,/home/fabric/.tokens.json
Project ID,6ce270de-788d-4e07-8bae-3206860a6387
Bastion Username,vsharan1_0000106410
Bastion Private Key File,/home/fabric/work/fabric_config/fabric_bastion_key
Bastion Host,bastion.fabric-testbed.net
Bastion Private Key Passphrase,None
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/slice_key
Slice Private Key Passphrase,None


## Create an Experiment Slice

The following creates three nodes, on different sites, with basic NICs connected to FABRIC's FABnetv4 internet. Sites can be changed, but make sure that the site is PTP-compatible. 

Current list of PTP-compatible sites:

- STAR
- MAX
- MICH
- MASS 
- UTAH
- NCSA
- UCSD
- FIU
- CLEM
- CERN

In [2]:
slice_name = 'PTP_slice_All'

#[site1,site2,site3] = fablib.get_random_sites(count=2, 
#        avoid=["DALL","GPN","LBNL","RENC","SALT","TACC","UKY","WASH","NCSA","LOSA","GATECH","INDI","MAX", "MASS","NEWY","SRI","UCSD"])

sites = ["STAR", "MAX", "MICH", "MASS", "UTAH", "NCSA", "UCSD", "FIU", "CLEM", "CERN"]
node_names  =  ["STAR", "MAX", "MICH", "MASS", "UTAH", "NCSA", "UCSD", "FIU", "CLEM", "CERN"]
print(node_names, sites)

['STAR', 'MAX', 'MICH', 'MASS', 'UTAH', 'NCSA', 'UCSD', 'FIU', 'CLEM', 'CERN'] ['STAR', 'MAX', 'MICH', 'MASS', 'UTAH', 'NCSA', 'UCSD', 'FIU', 'CLEM', 'CERN']


In [3]:
slice_name = "PTP_slice_All"

try:
    slice = fablib.get_slice(name=slice_name)
except Exception as e:
    print(f"Fail: {e}")
print (slice)

-----------  ------------------------------------
Slice Name   PTP_slice_All
Slice ID     c2c2c7fa-5edd-40e6-979e-282e204fb892
Slice State  ModifyError
Lease End    2023-11-19 06:01:27 +0000
-----------  ------------------------------------


In [4]:
#Create Slice. add_fabnet() automatically adds an L3 interface on each node and assigns an IP address.
slice = fablib.new_slice(name=slice_name)

nodes = []
for i in range(10):
    nodes.append(slice.add_node(name=node_names[i], site=sites[i], image='docker_rocky_8'))

for node in nodes:
    node.add_fabnet()

In [ ]:
slice.submit();

ID,c2c2c7fa-5edd-40e6-979e-282e204fb892
Name,PTP_slice_All
Lease Expiration (UTC),2023-11-19 06:01:27 +0000
Lease Start (UTC),2023-11-18 06:01:27 +0000
Project ID,6ce270de-788d-4e07-8bae-3206860a6387
State,StableError


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
a9af57bf-e4d3-4b30-8658-47649be370f4,CERN,2,8,10,docker_rocky_8,qcow2,cern-w1.fabric-testbed.net,CERN,rocky,2001:400:a100:3090:f816:3eff:fe54:7f7b,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config rocky@2001:400:a100:3090:f816:3eff:fe54:7f7b,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
44a89af0-0e5c-4b0f-921c-5531854aac63,CLEM,2,8,10,docker_rocky_8,qcow2,clem-w1.fabric-testbed.net,CLEM,rocky,2620:103:a006:12:f816:3eff:fed4:3565,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config rocky@2620:103:a006:12:f816:3eff:fed4:3565,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
c1052e28-e0d4-49f6-ae40-d850e4e8b945,FIU,2,8,10,docker_rocky_8,qcow2,fiu-w2.fabric-testbed.net,FIU,rocky,131.94.57.50,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config rocky@131.94.57.50,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
408eac1d-0434-4fef-bc3d-9e7d673d9170,MASS,2,8,10,docker_rocky_8,qcow2,mass-w3.fabric-testbed.net,MASS,rocky,205.172.170.95,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config rocky@205.172.170.95,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
afdb3ffa-f1fc-40fd-939a-c3d078fbcda8,MAX,2,8,10,docker_rocky_8,qcow2,max-w4.fabric-testbed.net,MAX,rocky,63.239.135.93,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config rocky@63.239.135.93,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
dd764526-56f7-4e82-9cb2-4b18866a6cfa,MICH,2,8,10,docker_rocky_8,qcow2,mich-w1.fabric-testbed.net,MICH,rocky,2607:f018:110:11:f816:3eff:fe2d:9d36,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config rocky@2607:f018:110:11:f816:3eff:fe2d:9d36,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
1099ac62-92e4-4184-a018-565a328c09c5,NCSA,2,8,10,docker_rocky_8,qcow2,ncsa-w1.fabric-testbed.net,NCSA,rocky,2620:0:c80:1001:f816:3eff:fe76:b70f,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config rocky@2620:0:c80:1001:f816:3eff:fe76:b70f,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
a7187a86-65f2-424a-90b3-db6e8aa6f6b2,STAR,2,8,10,docker_rocky_8,qcow2,star-w1.fabric-testbed.net,STAR,rocky,2001:400:a100:3030:f816:3eff:feff:b52c,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config rocky@2001:400:a100:3030:f816:3eff:feff:b52c,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
c7173e6f-a608-45c9-b206-0b0949298e37,UCSD,2,8,10,docker_rocky_8,qcow2,ucsd-w4.fabric-testbed.net,UCSD,rocky,132.249.252.186,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config rocky@132.249.252.186,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
144552c6-47d9-4383-98d0-c7ef66a4209b,UTAH,2,8,10,docker_rocky_8,qcow2,utah-w4.fabric-testbed.net,UTAH,rocky,2001:1948:417:7:f816:3eff:fe91:6a72,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config rocky@2001:1948:417:7:f816:3eff:fe91:6a72,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
eca967b3-2ac6-4c99-9c3a-20709547761b,FABNET_IPv4_CERN,L3,FABNetv4,CERN,FABNET_IPv4_CERN.subnet,FABNET_IPv4_CERN.gateway,Active,
a9602627-ac96-413e-9d2f-d32535764765,FABNET_IPv4_CLEM,L3,FABNetv4,CLEM,FABNET_IPv4_CLEM.subnet,FABNET_IPv4_CLEM.gateway,Failed,"failed lease update- all units failed priming: Exception during create for unit: a9602627-ac96-413e-9d2f-d32535764765 Playbook has failed tasks: NSO commit returned JSON-RPC error: type: rpc.method.failed, code: -32000, message: Method failed, data: message: Network Element Driver: device clem-data-sw: out of sync, internal: jsonrpc_tx_commit357#all units failed priming: Exception during create for unit: a9602627-ac96-413e-9d2f-d32535764765 Playbook has failed tasks: NSO commit returned JSON-RPC error: type: rpc.method.failed, code: -32000, message: Method failed, data: message: Network Element Driver: device clem-data-sw: out of sync, internal: jsonrpc_tx_commit357#"
af2e6052-892f-4400-a59b-2455d82734fb,FABNET_IPv4_FIU,L3,FABNetv4,FIU,FABNET_IPv4_FIU.subnet,FABNET_IPv4_FIU.gateway,Active,
000dd82d-53b3-4a0c-bd3c-690022fa7bb8,FABNET_IPv4_MASS,L3,FABNetv4,MASS,FABNET_IPv4_MASS.subnet,FABNET_IPv4_MASS.gateway,Active,
da068637-f792-461a-8b9c-a9c380804612,FABNET_IPv4_MAX,L3,FABNetv4,MAX,FABNET_IPv4_MAX.subnet,FABNET_IPv4_MAX.gateway,Active,
71b81cfb-15c1-4c9d-b267-65127e3f9944,FABNET_IPv4_MICH,L3,FABNetv4,MICH,FABNET_IPv4_MICH.subnet,FABNET_IPv4_MICH.gateway,Active,
92fa8ea7-6ad0-4c47-bdb2-5e58b32ac8dc,FABNET_IPv4_NCSA,L3,FABNetv4,NCSA,FABNET_IPv4_NCSA.subnet,FABNET_IPv4_NCSA.gateway,Active,
f7e1f70c-98d5-47c3-947e-8c362350b330,FABNET_IPv4_STAR,L3,FABNetv4,STAR,FABNET_IPv4_STAR.subnet,FABNET_IPv4_STAR.gateway,Active,
3464c979-e373-4014-aaa4-83d2c737bea8,FABNET_IPv4_UCSD,L3,FABNetv4,UCSD,FABNET_IPv4_UCSD.subnet,FABNET_IPv4_UCSD.gateway,Closed,"failed lease update- all units failed priming: Exception during create for unit: 3464c979-e373-4014-aaa4-83d2c737bea8 Playbook has failed tasks: NSO commit returned JSON-RPC error: type: rpc.method.failed, code: -32000, message: Method failed, data: message: External error in the NED implementation for device ucsd-data-sw: Sat Nov 18 06:05:52.063 UTCrnrn Failed to commit one or more configuration items during a pseudo-atomic operation. All changes made have been reverted.rn SEMANTIC ERRORS: This configuration was rejected by rn the system due to semantic errors. The individual rn errors with each failed configuration command can be rn found below.rnrnrnl2vpnrn bridge group bg-FABNET_IPv4_UCSD-3464c979-e3rn bridge-domain bd-FABNET_IPv4_UCSD-3464c97rn interface HundredGigE0/0/0/11.2012rn Invalid argument: Interface already used for L2VPNrn rn rn rnrnend, internal: jsonrpc_tx_commit357#all units failed priming: Exception during create for unit: 3464c979-e373-4014-aaa4-83d2c737bea8 Playbook has failed tasks: NSO commit returned JSON-RPC error: type: rpc.method.failed, code: -32000, message: Method failed, data: message: External error in the NED implementation for device ucsd-data-sw: Sat Nov 18 06:05:52.063 UTCrnrn Failed to commit one or more configuration items during a pseudo-atomic operation. All changes made have been reverted.rn SEMANTIC ERRORS: This configuration was rejected by rn the system due to semantic errors. The individual rn errors with each failed configuration command can be rn found below.rnrnrnl2vpnrn bridge group bg-FABNET_IPv4_UCSD-3464c979-e3rn bridge-domain bd-FABNET_IPv4_UCSD-3464c97rn interface HundredGigE0/0/0/11.2012rn Invalid argument: Interface already used for L2VPNrn rn rn rnrnend, internal: jsonrpc_tx_commit357#"
04c21049-9c3f-4786-b8c2-2800178fe4cd,FABNET_IPv4_UTAH,L3,FABNetv4,UTAH,FABNET_IPv4_UTAH.subnet,FABNET_IPv4_UTAH.gateway,Active,



Time to stable 464 seconds
Running post_boot_config ... 
Running post boot config threads ...
Post boot config FIU, Done! (12 sec)
Post boot config MASS, Done! (12 sec)
Post boot config UTAH, Done! (12 sec)
Post boot config MICH, Done! (13 sec)
Post boot config CLEM, Done! (13 sec)
Post boot config UCSD, Done! (13 sec)
Post boot config NCSA, Done! (13 sec)
Post boot config MAX, Done! (13 sec)
Post boot config STAR, Done! (14 sec)
Post boot config CERN, Done! (16 sec)
Saving fablib data... 

## (Optional) Observe the Slice's Attributes

In [5]:
try:
    slice = fablib.get_slice(name=slice_name)
    slice.show()
    slice.list_nodes()
    slice.list_networks()
    slice.list_interfaces()
except Exception as e:
    print(f"Exception: {e}")

ID,c2c2c7fa-5edd-40e6-979e-282e204fb892
Name,PTP_slice_All
Lease Expiration (UTC),2023-11-19 06:01:27 +0000
Lease Start (UTC),2023-11-18 06:01:27 +0000
Project ID,6ce270de-788d-4e07-8bae-3206860a6387
State,ModifyError


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
a9af57bf-e4d3-4b30-8658-47649be370f4,CERN,2,8,10,docker_rocky_8,qcow2,cern-w1.fabric-testbed.net,CERN,rocky,2001:400:a100:3090:f816:3eff:fe54:7f7b,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config rocky@2001:400:a100:3090:f816:3eff:fe54:7f7b,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
44a89af0-0e5c-4b0f-921c-5531854aac63,CLEM,2,8,10,docker_rocky_8,qcow2,clem-w1.fabric-testbed.net,CLEM,rocky,2620:103:a006:12:f816:3eff:fed4:3565,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config rocky@2620:103:a006:12:f816:3eff:fed4:3565,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
c1052e28-e0d4-49f6-ae40-d850e4e8b945,FIU,2,8,10,docker_rocky_8,qcow2,fiu-w2.fabric-testbed.net,FIU,rocky,131.94.57.50,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config rocky@131.94.57.50,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
408eac1d-0434-4fef-bc3d-9e7d673d9170,MASS,2,8,10,docker_rocky_8,qcow2,mass-w3.fabric-testbed.net,MASS,rocky,205.172.170.95,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config rocky@205.172.170.95,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
afdb3ffa-f1fc-40fd-939a-c3d078fbcda8,MAX,2,8,10,docker_rocky_8,qcow2,max-w4.fabric-testbed.net,MAX,rocky,63.239.135.93,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config rocky@63.239.135.93,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
dd764526-56f7-4e82-9cb2-4b18866a6cfa,MICH,2,8,10,docker_rocky_8,qcow2,mich-w1.fabric-testbed.net,MICH,rocky,2607:f018:110:11:f816:3eff:fe2d:9d36,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config rocky@2607:f018:110:11:f816:3eff:fe2d:9d36,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
1099ac62-92e4-4184-a018-565a328c09c5,NCSA,2,8,10,docker_rocky_8,qcow2,ncsa-w1.fabric-testbed.net,NCSA,rocky,2620:0:c80:1001:f816:3eff:fe76:b70f,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config rocky@2620:0:c80:1001:f816:3eff:fe76:b70f,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
a7187a86-65f2-424a-90b3-db6e8aa6f6b2,STAR,2,8,10,docker_rocky_8,qcow2,star-w1.fabric-testbed.net,STAR,rocky,2001:400:a100:3030:f816:3eff:feff:b52c,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config rocky@2001:400:a100:3030:f816:3eff:feff:b52c,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
c7173e6f-a608-45c9-b206-0b0949298e37,UCSD,2,8,10,docker_rocky_8,qcow2,ucsd-w4.fabric-testbed.net,UCSD,rocky,132.249.252.186,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config rocky@132.249.252.186,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
144552c6-47d9-4383-98d0-c7ef66a4209b,UTAH,2,8,10,docker_rocky_8,qcow2,utah-w4.fabric-testbed.net,UTAH,rocky,2001:1948:417:7:f816:3eff:fe91:6a72,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config rocky@2001:1948:417:7:f816:3eff:fe91:6a72,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
eca967b3-2ac6-4c99-9c3a-20709547761b,FABNET_IPv4_CERN,L3,FABNetv4,CERN,10.143.2.0/24,10.143.2.1,Active,
a9602627-ac96-413e-9d2f-d32535764765,FABNET_IPv4_CLEM,L3,FABNetv4,CLEM,10.136.2.0/24,10.136.2.1,Failed,"failed lease update- all units failed priming: Exception during create for unit: a9602627-ac96-413e-9d2f-d32535764765 Playbook has failed tasks: NSO commit returned JSON-RPC error: type: rpc.method.failed, code: -32000, message: Method failed, data: message: Network Element Driver: device clem-data-sw: out of sync, internal: jsonrpc_tx_commit357#all units failed priming: Exception during create for unit: a9602627-ac96-413e-9d2f-d32535764765 Playbook has failed tasks: NSO commit returned JSON-RPC error: type: rpc.method.failed, code: -32000, message: Method failed, data: message: Network Element Driver: device clem-data-sw: out of sync, internal: jsonrpc_tx_commit357#"
af2e6052-892f-4400-a59b-2455d82734fb,FABNET_IPv4_FIU,L3,FABNetv4,FIU,10.135.133.0/24,10.135.133.1,Active,
000dd82d-53b3-4a0c-bd3c-690022fa7bb8,FABNET_IPv4_MASS,L3,FABNetv4,MASS,10.131.132.0/24,10.131.132.1,Active,
da068637-f792-461a-8b9c-a9c380804612,FABNET_IPv4_MAX,L3,FABNetv4,MAX,10.130.7.0/24,10.130.7.1,Active,
71b81cfb-15c1-4c9d-b267-65127e3f9944,FABNET_IPv4_MICH,L3,FABNetv4,MICH,10.131.3.0/24,10.131.3.1,Active,
92fa8ea7-6ad0-4c47-bdb2-5e58b32ac8dc,FABNET_IPv4_NCSA,L3,FABNetv4,NCSA,10.132.131.0/24,10.132.131.1,Active,
f7e1f70c-98d5-47c3-947e-8c362350b330,FABNET_IPv4_STAR,L3,FABNetv4,STAR,10.129.129.0/24,10.129.129.1,Active,
3464c979-e373-4014-aaa4-83d2c737bea8,FABNET_IPv4_UCSD,L3,FABNetv4,UCSD,10.134.130.0/24,10.134.130.1,Closed,"failed lease update- all units failed priming: Exception during create for unit: 3464c979-e373-4014-aaa4-83d2c737bea8 Playbook has failed tasks: NSO commit returned JSON-RPC error: type: rpc.method.failed, code: -32000, message: Method failed, data: message: External error in the NED implementation for device ucsd-data-sw: Sat Nov 18 06:05:52.063 UTCrnrn Failed to commit one or more configuration items during a pseudo-atomic operation. All changes made have been reverted.rn SEMANTIC ERRORS: This configuration was rejected by rn the system due to semantic errors. The individual rn errors with each failed configuration command can be rn found below.rnrnrnl2vpnrn bridge group bg-FABNET_IPv4_UCSD-3464c979-e3rn bridge-domain bd-FABNET_IPv4_UCSD-3464c97rn interface HundredGigE0/0/0/11.2012rn Invalid argument: Interface already used for L2VPNrn rn rn rnrnend, internal: jsonrpc_tx_commit357#all units failed priming: Exception during create for unit: 3464c979-e373-4014-aaa4-83d2c737bea8 Playbook has failed tasks: NSO commit returned JSON-RPC error: type: rpc.method.failed, code: -32000, message: Method failed, data: message: External error in the NED implementation for device ucsd-data-sw: Sat Nov 18 06:05:52.063 UTCrnrn Failed to commit one or more configuration items during a pseudo-atomic operation. All changes made have been reverted.rn SEMANTIC ERRORS: This configuration was rejected by rn the system due to semantic errors. The individual rn errors with each failed configuration command can be rn found below.rnrnrnl2vpnrn bridge group bg-FABNET_IPv4_UCSD-3464c979-e3rn bridge-domain bd-FABNET_IPv4_UCSD-3464c97rn interface HundredGigE0/0/0/11.2012rn Invalid argument: Interface already used for L2VPNrn rn rn rnrnend, internal: jsonrpc_tx_commit357#"
04c21049-9c3f-4786-b8c2-2800178fe4cd,FABNET_IPv4_UTAH,L3,FABNetv4,UTAH,10.132.5.0/24,10.132.5.1,Active,


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node
STAR-FABNET_IPv4_STAR_nic-p1,p1,STAR,FABNET_IPv4_STAR,100,auto,,4E:10:D1:27:F2:48,enp7s0,enp7s0,10.129.129.2,6
MAX-FABNET_IPv4_MAX_nic-p1,p1,MAX,FABNET_IPv4_MAX,100,auto,,06:3B:31:3D:E8:28,enp7s0,enp7s0,10.130.7.2,4
MICH-FABNET_IPv4_MICH_nic-p1,p1,MICH,FABNET_IPv4_MICH,100,auto,,22:45:42:BC:5D:36,enp7s0,enp7s0,10.131.3.2,6
MASS-FABNET_IPv4_MASS_nic-p1,p1,MASS,FABNET_IPv4_MASS,100,auto,,06:51:15:6C:19:23,enp7s0,enp7s0,10.131.132.2,4
UTAH-FABNET_IPv4_UTAH_nic-p1,p1,UTAH,FABNET_IPv4_UTAH,100,auto,,16:4A:C8:88:50:28,enp7s0,enp7s0,10.132.5.2,4
NCSA-FABNET_IPv4_NCSA_nic-p1,p1,NCSA,FABNET_IPv4_NCSA,100,auto,,02:F6:02:BD:15:CE,enp7s0,enp7s0,10.132.131.2,6
UCSD-FABNET_IPv4_UCSD_nic-p1,p1,UCSD,FABNET_IPv4_UCSD,100,auto,,02:18:C1:EE:87:8F,enp6s0,enp6s0,10.134.130.2,4
FIU-FABNET_IPv4_FIU_nic-p1,p1,FIU,FABNET_IPv4_FIU,100,auto,,02:9E:15:F7:A7:E5,enp6s0,enp6s0,10.135.133.2,6
CLEM-FABNET_IPv4_CLEM_nic-p1,p1,CLEM,FABNET_IPv4_CLEM,100,auto,,06:F7:67:38:E7:26,enp6s0,enp6s0,10.136.2.2,6
CERN-FABNET_IPv4_CERN_nic-p1,p1,CERN,FABNET_IPv4_CERN,100,auto,,0A:BE:C1:46:1D:4C,enp7s0,enp7s0,10.143.2.2,6


<!-- ## Configure IP Addresses

### Get the Assigned Subnet

FABnetv4 networks are assigned a subnet and gateway by FABRIC.  You can get the subnet and available IPs from the FABlib objects.  -->

## Check connectivity via Experimenter's network

In [6]:
try:
    nodes[0] = slice.get_node(name=node_names[0])   
    nodes[2] = slice.get_node(name=node_names[2])
    node3_addr = nodes[2].get_interface(network_name=f'FABNET_IPv4_{nodes[2].get_site()}').get_ip_addr()
    
    nodes[0].execute('hostname')
    stdout, stderr = nodes[0].execute(f'ping -c 5 {node3_addr}')  
    
except Exception as e:
    print(f"Exception: {e}")

STAR
PING 10.131.3.2 (10.131.3.2) 56(84) bytes of data.
64 bytes from 10.131.3.2: icmp_seq=1 ttl=62 time=7.21 ms
64 bytes from 10.131.3.2: icmp_seq=2 ttl=62 time=5.28 ms
64 bytes from 10.131.3.2: icmp_seq=3 ttl=62 time=5.29 ms
64 bytes from 10.131.3.2: icmp_seq=4 ttl=62 time=5.29 ms
64 bytes from 10.131.3.2: icmp_seq=5 ttl=62 time=5.28 ms

--- 10.131.3.2 ping statistics ---
5 packets transmitted, 5 received, 0% packet loss, time 4005ms
rtt min/avg/max/mdev = 5.277/5.667/7.206/0.769 ms


# Prepare each node for time precision experiments

In [7]:
nodes = slice.get_nodes()

## Enable NAT64

### Upload and Execute the NAT64 Script

We will use a NAT64 script, which configures an *IPv6* node so that it can access non-*IPv6* services. To view the script, click [here](./nat64.sh). This script sets up the [Public NAT64 Service](https://nat64.net) on your node.

For more details on how uploading and executing scripts works on FABRIC, click [here](../upload_and_execute/upload_and_execute.ipynb).

Based on https://nat64.net/
Note that NAT64 could go away at any minute, as it is a public service ran independently by Kasper Dupont, and is not affiliated with FABRIC.

In [8]:
# Change this to all nodes

from ipaddress import ip_address, IPv6Address

for node in nodes:
    try:     
        # If the node is an IPv6 Node then configure NAT64
        if node.validIPAddress(node.get_management_ip()) == "IPv6":
            # needed to fix sudo unable to resolve error
            commands = """
            sudo echo -n "127.0.0.1 " | sudo cat - /etc/hostname  | sudo tee -a /etc/hosts;
            sudo echo -n "2a01:4f9:c010:3f02:64:0:8c52:7103       github.com\n"|sudo tee -a /etc/hosts;
            sudo echo -n "2a01:4f9:c010:3f02:64:0:8c52:7009       codeload.github.com\n"|sudo tee -a /etc/hosts;
            sudo echo -n "2a01:4f9:c010:3f02:64:0:b9c7:6e85       objects.githubusercontent.com\n"|sudo tee -a /etc/hosts;
            sudo echo -n "2600:1fa0:80b4:db49:34d9:6d1e::         ansible-galaxy.s3.amazonaws.com\n"|sudo tee -a /etc/hosts;
            sudo echo -n "2a01:4f9:c010:3f02:64:0:3455:9777       packages.confluent.io\n"|sudo tee -a /etc/hosts;
            """
            stdout, stderr = node.execute(commands, quiet=True)

        # Access non-IPv6 Services
        stdout, stderr = node.execute(f'sudo yum install -y -q git && git clone https://github.com/fabric-testbed/jupyter-examples.git')

        stdout, stderr = node.execute(f'ls jupyter-examples')

    except Exception as e:
        print(f"Exception: {e}")

 Cloning into 'jupyter-examples'...
 Changelog.md
configure.ipynb
docker_containers
fabric_examples
fabric_ssh_tunnel_tools
LICENSE
Readme.md
requirements.txt
start_here.ipynb
 Cloning into 'jupyter-examples'...
 Changelog.md
configure.ipynb
docker_containers
fabric_examples
fabric_ssh_tunnel_tools
LICENSE
Readme.md
requirements.txt
start_here.ipynb
 Cloning into 'jupyter-examples'...
 Changelog.md
configure.ipynb
docker_containers
fabric_examples
fabric_ssh_tunnel_tools
LICENSE
Readme.md
requirements.txt
start_here.ipynb
 Cloning into 'jupyter-examples'...
 Changelog.md
configure.ipynb
docker_containers
fabric_examples
fabric_ssh_tunnel_tools
LICENSE
Readme.md
requirements.txt
start_here.ipynb
 Cloning into 'jupyter-examples'...
 Changelog.md
configure.ipynb
docker_containers
fabric_examples
fabric_ssh_tunnel_tools
LICENSE
Readme.md
requirements.txt
start_here.ipynb
 Cloning into 'jupyter-examples'...
 Changelog.md
configure.ipynb
docker_containers
fabric_examples
fabric_ssh_tunnel_to

## Set up PTP (Precision Time Protocol)

#### Clone MeasurementFramework Git repo Locally

In [9]:
%%bash
rm -rf /tmp/MF;
git clone https://github.com/fabric-testbed/MeasurementFramework.git /tmp/MF;

Cloning into '/tmp/MF'...


#### Install and setup linux ptp package on all nodes 

In [10]:
pre_requisites = f"""
`sudo apt-get update;sudo apt-get -y install ansible git` || `sudo dnf -y install epel-release ;sudo dnf -y install ansible git`
"""
ansible_instructions = f"""
cd /tmp/ansible;ansible-playbook --connection=local --inventory 127.0.0.1, --limit 127.0.0.1 playbook_fabric_experiment_ptp.yml;
"""

for node in nodes:
    print (f"Installing PTP on {node.get_name()}")
    node.upload_directory('/tmp/MF/instrumentize/ptp/ansible','/tmp/')
    node.execute(f"{pre_requisites}"\
                 f"{ansible_instructions}"\
                )
    print (f"Installation of PTP Completed on {node.get_name()}\n\n")

Installing PTP on STAR

PLAY [all] *********************************************************************
 sudo: apt-get: command not found
sudo: apt-get: command not found
Importing GPG key 0xE451E5B5:
 Userid     : "CentOS Storage SIG (http://wiki.centos.org/SpecialInterestGroup/Storage) <security@centos.org>"
 Fingerprint: 7412 9C0B 173B 071A 3775 951A D4A2 E50B E451 E5B5
 From       : /etc/pki/rpm-gpg/RPM-GPG-KEY-CentOS-SIG-Storage
bash: line 1: Last: command not found
 
TASK [Gathering Facts] *********************************************************
ok: [127.0.0.1]

TASK [linuxptp : include] ******************************************************
included: /tmp/ansible/roles/linuxptp/tasks/find_ptp_interfaces.yml for 127.0.0.1
included: /tmp/ansible/roles/linuxptp/tasks/find_ptp_interfaces.yml for 127.0.0.1
included: /tmp/ansible/roles/linuxptp/tasks/find_ptp_interfaces.yml for 127.0.0.1

TASK [linuxptp : Check if Interface lo is PTP capable] *************************
changed: [127.

## Start Docker 

In [11]:
for node in nodes:
    node.execute("sudo systemctl start docker")
    node.execute("sudo systemctl enable docker")
    node.execute("sudo usermod -aG docker rocky")
    
    print(f"\n Verify installtion... on {node.get_name()}")
    node.execute("docker --help")

 Created symlink /etc/systemd/system/multi-user.target.wants/docker.service → /usr/lib/systemd/system/docker.service.
 
 Verify installtion... on STAR

Usage:  docker [OPTIONS] COMMAND

A self-sufficient runtime for containers

Common Commands:
  run         Create and run a new container from an image
  exec        Execute a command in a running container
  ps          List containers
  build       Build an image from a Dockerfile
  pull        Download an image from a registry
  push        Upload an image to a registry
  images      List images
  login       Log in to a registry
  logout      Log out from a registry
  search      Search Docker Hub for images
  version     Show the Docker version information
  info        Display system-wide information

Management Commands:
  builder     Manage builds
  buildx*     Docker Buildx (Docker Inc., v0.10.5)
  checkpoint  Manage checkpoints
  compose*    Docker Compose (Docker Inc., v2.18.1)
  container   Manage containers
  context     Ma

# (Optional) Extend the slice (Add 14 days)

In [12]:
try:
    slice = fablib.get_slice(name=slice_name)
    print(f"Lease End         : {slice.get_lease_end()}")
       
except Exception as e:
    print(f"Exception: {e}")

Lease End         : 2023-11-19 06:01:27 +0000


In [13]:
import datetime

#Extend slice
end_date = (datetime.datetime.now().astimezone() + datetime.timedelta(days=14)).strftime("%Y-%m-%d %H:%M:%S %z")

try:
    slice = fablib.get_slice(name=slice_name)
    slice.renew(end_date)
    print(f"New lease end date : {slice.get_lease_end()}")
    
except Exception as e:
    print(f"Exception: {e}")

New lease end date : 2023-11-19 06:01:27 +0000


# Delete the Slice

In [4]:
"""
try:
    slice = fablib.get_slice(name=slice_name)
    slice.delete()
except Exception as e:
    print(f"Exception: {e}")
"""

# Run OWL (One-Way Latency Measurements) 

Script for running OWL on one or more nodes in a slice. Output for each measurement is saved as a pcap file on the destination node. 
Current implementation on IPv4 experimenter interfaces only. Assumes each node has only 1 experimental interface.

(Tested on 2023/08/18)

# First things first

In [14]:
from fabrictestbed_extensions.fablib.fablib import fablib
import json

In [15]:
slice_name = "PTP_slice_All"

try:
    slice = fablib.get_slice(name=slice_name)
except Exception as e:
    print(f"Fail: {e}")
print (slice)

-----------  ------------------------------------
Slice Name   PTP_slice_All
Slice ID     c2c2c7fa-5edd-40e6-979e-282e204fb892
Slice State  ModifyError
Lease End    2023-12-02 07:16:54 +0000
-----------  ------------------------------------


In [18]:
from mflib import owl

# for debugging only
#import owl_local as owl

In [19]:
try:    
    node1 = slice.get_node(name=node_names[0])
    node2 = slice.get_node(name=node_names[1])
    node3 = slice.get_node(name=node_names[2])
    nodes = slice.get_nodes()
except Exception as e:
    print(f"Fail: {e}")   

## Look up IPs of nodes and decide which IP address to use 

Checking experiment IP addresses by filtering out meas_network addresses. Asssumes one experimenter interface per node.

In [20]:
node_ip_list = owl.nodes_ip_addrs(slice)
print(node_ip_list)

{'STAR': '10.129.129.2', 'MAX': '10.130.7.2', 'MICH': '10.131.3.2', 'MASS': '10.131.132.2', 'UTAH': '10.132.5.2', 'NCSA': '10.132.131.2', 'UCSD': '10.134.130.2', 'FIU': '10.135.133.2', 'CLEM': '10.136.2.2', 'CERN': '10.143.2.2'}


## Check OWL prerequisites on all nodes in the slice

Runs the following test and prints output on each node:

+ `git clone`
+ `ps -ef | grep phc2sys`
+ `docker --help`

In [21]:
owl.check_owl_prerequisites(slice)


***** On STAR...

***** Is github reachable?
 Cloning into 'teaching-materials'...
 total 8
drwxrwxr-x. 6 rocky rocky 4096 Nov 18 06:13 jupyter-examples
drwxrwxr-x. 5 rocky rocky 4096 Nov 18 07:22 teaching-materials

***** Is PTP is enabled?
root       16914       1  0 06:47 ?        00:00:00 /usr/sbin/phc2sys -u 8 -E linreg -O +0 -s /dev/ptp1 -c enp7s0
root       17412       1  0 06:47 ?        00:00:00 /usr/sbin/phc2sys -u 8 -E linreg -O +0 -s /dev/ptp1 -c CLOCK_REALTIME
rocky      26918   26917  0 07:22 ?        00:00:00 bash -c ps -ef | grep phc2sys
rocky      26938   26918  0 07:22 ?        00:00:00 grep phc2sys

*****Is Docker installed?

Usage:  docker [OPTIONS] COMMAND

A self-sufficient runtime for containers

Common Commands:
  run         Create and run a new container from an image
  exec        Execute a command in a running container
  ps          List containers
  build       Build an image from a Dockerfile

***** On MAX...

***** Is github reachable?
 Cloning into 'te

## Pull OWL image from DockerHub

In [22]:
image_name="fabrictestbed/owl:0.1.4"

for node in nodes:
    node.execute(f"sudo docker pull {image_name}") 

0.1.4: Pulling from fabrictestbed/owl
8b91b88d5577: Pulling fs layer
824416e23423: Pulling fs layer
8d53da260408: Pulling fs layer
84c8c79126f6: Pulling fs layer
2e1c130fa3ec: Pulling fs layer
e9571e66d363: Pulling fs layer
c2426a8d87d1: Pulling fs layer
5922e7f17bf6: Pulling fs layer
7935dece71a7: Pulling fs layer
aa4bd601fd3f: Pulling fs layer
6d94b93994d0: Pulling fs layer
e1e2036205d2: Pulling fs layer
895f77b56ce1: Pulling fs layer
84c8c79126f6: Waiting
2e1c130fa3ec: Waiting
c2426a8d87d1: Waiting
e9571e66d363: Waiting
5922e7f17bf6: Waiting
7935dece71a7: Waiting
6d94b93994d0: Waiting
e1e2036205d2: Waiting
aa4bd601fd3f: Waiting
895f77b56ce1: Waiting
824416e23423: Verifying Checksum
824416e23423: Download complete
8d53da260408: Verifying Checksum
8d53da260408: Download complete
84c8c79126f6: Verifying Checksum
84c8c79126f6: Download complete
2e1c130fa3ec: Verifying Checksum
2e1c130fa3ec: Download complete
8b91b88d5577: Verifying Checksum
8b91b88d5577: Download complete
c2426a8d87d1: 

# Run the experiment

There are 3 ways to use OWL

1. Specify each link with sender-capturer pair: this should be easiest method for most users.
2. Start each sender and capturer manually: this is useful when you want to have more precise control
3. Start on all links: this starts OWL on every pair of nodes in the slice (excluding meas-node if there is one)


**Note: By default, all the previously saved pcap files will be deleted when a node starts a new OWL capturer session. If this behavior is not desired, set `delete_previous_output` to `False`.**

## Method 1: Specify links by source and destination nodes
This examples runs OWL on the following links:

+ Node1 --> Node2
+ Node2 --> Node3
+ Node3 --> Node2

For each link, sender and capturer containers will start. If one node is a destination for more than 1 sender, only 1 capturer container instance will be created.

In [ ]:
owl.start_owl(slice, src_node=slice.get_node(name='Node1'), dst_node=slice.get_node(name='Node2'), img_name=image_name, probe_freq=1, no_ptp=False, outfile=None, 
              duration = 120, delete_previous_output=True)

In [ ]:
owl.start_owl(slice, src_node=slice.get_node(name='Node2'), dst_node=slice.get_node(name='Node3'), img_name=image_name, probe_freq=1, no_ptp=False, outfile=None, 
              duration = 600, delete_previous_output=True)

In [ ]:
owl.start_owl(slice, src_node=slice.get_node(name='Node3'), dst_node=slice.get_node(name='Node2'), img_name=image_name, probe_freq=1, no_ptp=False, outfile=None, 
              duration = 600, delete_previous_output=True)

### Check if they are running

In [ ]:
owl.check_owl_all(slice)

### Stop all OWL containers

In [ ]:
owl.stop_owl_all(slice)

### Verify

In [ ]:
owl.check_owl_all(slice)

## Method 2: start each sender, caputrer separately

If user desires more granular control, sender and capturer can be started/stopped independently.


In [ ]:
owl.start_owl_sender(slice, src_node=slice.get_node(name='Node1'), dst_node=slice.get_node(name='Node2'), img_name=image_name, duration=120)

In [ ]:
owl.start_owl_sender(slice, src_node=slice.get_node(name='Node1'), dst_node=slice.get_node(name='Node3'), img_name=image_name, duration=120)

In [ ]:
owl.start_owl_capturer(slice,  dst_node=slice.get_node(name='Node2'), img_name=image_name, duration=120)

In [ ]:
owl.start_owl_sender(slice, src_node=slice.get_node(name='Node3'), dst_node=slice.get_node(name='Node2'), img_name=image_name, duration=120)

### Check how the containers are running

In [ ]:
owl.check_owl_all(slice)

### Stop OWL containers on each node

The method finds all containers with the prefix `owl-sender_` in their names, stops and removes them.

In [ ]:
owl.stop_owl_sender(slice, src_node=slice.get_node(name='Node1'), dst_node=slice.get_node(name='Node2'))

In [ ]:
owl.stop_owl_sender(slice, src_node=slice.get_node(name='Node1'), dst_node=slice.get_node(name='Node3'))

In [ ]:
owl.stop_owl_capturer(slice,  dst_node=slice.get_node(name='Node2'))

In [ ]:
owl.stop_owl_sender(slice, src_node=slice.get_node(name='Node3'), dst_node=slice.get_node(name='Node2'))

### Verify

In [ ]:
owl.check_owl_all(slice)

## Method 3: start on all links

If OWL measurements are desired on ALL possible links, call the following method. It will start sender and capturer containers on all nodes with each node sending probe packets to all the rest of the nodes in the slice.

In [26]:
owl.start_owl_all(slice, img_name=image_name, probe_freq=1, outfile=None, duration=600, delete_previous=True)

STAR --> MAX
Staring sender
7c5c41c871a199bba4b6c024ca8dfd473ddb795da49bbd0fd974ab51217d3c92
Staring capturer
owl-capturer_10.130.7.2
capturer already running on MAX
STAR --> MICH
Staring sender
3bf3b60aeda927658858bd7051934fbad4569c6d17c6a75cc3dabd6728341025
Staring capturer
owl-capturer_10.131.3.2
capturer already running on MICH
STAR --> MASS
Staring sender
3461ff322837d7d879f20a4269a0c088df9611c2a675aac1b2d29f5fb82ea95b
Staring capturer
owl-capturer_10.131.132.2
capturer already running on MASS
STAR --> UTAH
Staring sender
4a57fadf1e835ed22f15425fdf64b7f4e9c1438ad7f742734ddc6bd211e19599
Staring capturer
owl-capturer_10.132.5.2
capturer already running on UTAH
STAR --> NCSA
Staring sender
fac539dc8c903a81bda0fcdc6c2f4b44c4861157e616ae4f5473c7f1d66a1bfc
Staring capturer
owl-capturer_10.132.131.2
capturer already running on NCSA
STAR --> UCSD
Staring sender
18ee088eee66f4bba69d7a912a75aa5c61d5b6f9e2d0524d56cb25ec882123ae
Staring capturer
owl-capturer_10.134.130.2
capturer already runn

### Check if they are running

In [24]:
owl.check_owl_all(slice)

STAR
CONTAINER ID   IMAGE                     COMMAND                  CREATED          STATUS          PORTS     NAMES
14059ab95863   fabrictestbed/owl:0.1.4   "python3 sock_ops/ud…"   43 minutes ago   Up 43 minutes             owl-capturer_10.129.129.2
MAX
CONTAINER ID   IMAGE                     COMMAND                  CREATED          STATUS          PORTS     NAMES
6db9673a0d8f   fabrictestbed/owl:0.1.4   "python3 sock_ops/ud…"   49 minutes ago   Up 49 minutes             owl-capturer_10.130.7.2
MICH
CONTAINER ID   IMAGE                     COMMAND                  CREATED          STATUS          PORTS     NAMES
c9bbaec947b0   fabrictestbed/owl:0.1.4   "python3 sock_ops/ud…"   48 minutes ago   Up 48 minutes             owl-capturer_10.131.3.2
MASS
CONTAINER ID   IMAGE                     COMMAND                  CREATED          STATUS          PORTS     NAMES
76720c534357   fabrictestbed/owl:0.1.4   "python3 sock_ops/ud…"   48 minutes ago   Up 48 minutes             owl-capture

### Stop all OWL containers

In [ ]:
owl.stop_owl_all(slice)

### Verify

In [27]:
owl.check_owl_all(slice)

STAR
CONTAINER ID   IMAGE                     COMMAND                  CREATED       STATUS       PORTS     NAMES
14059ab95863   fabrictestbed/owl:0.1.4   "python3 sock_ops/ud…"   2 hours ago   Up 2 hours             owl-capturer_10.129.129.2
MAX
CONTAINER ID   IMAGE                     COMMAND                  CREATED       STATUS       PORTS     NAMES
6db9673a0d8f   fabrictestbed/owl:0.1.4   "python3 sock_ops/ud…"   2 hours ago   Up 2 hours             owl-capturer_10.130.7.2
MICH
CONTAINER ID   IMAGE                     COMMAND                  CREATED       STATUS       PORTS     NAMES
c9bbaec947b0   fabrictestbed/owl:0.1.4   "python3 sock_ops/ud…"   2 hours ago   Up 2 hours             owl-capturer_10.131.3.2
MASS
CONTAINER ID   IMAGE                     COMMAND                  CREATED       STATUS       PORTS     NAMES
76720c534357   fabrictestbed/owl:0.1.4   "python3 sock_ops/ud…"   2 hours ago   Up 2 hours             owl-capturer_10.131.132.2
UTAH
CONTAINER ID   IMAGE        

# Get the Output files

By default, all the pcap files are saved in `/home/rocky/owl-output` as `{dest_ip}.pcap` on the desination node.

## Check the availability of output files

In [28]:
owl_output_dir = '/home/rocky/owl-output'

for node in nodes:
    print(node.get_name())
    node.execute(f"ls -lh {owl_output_dir}")

STAR
total 560K
-rw-r--r--. 1 root root 554K Nov 18 09:16 10.129.129.2.pcap
MAX
total 556K
-rw-r--r--. 1 root root 550K Nov 18 09:16 10.130.7.2.pcap
MICH
total 556K
-rw-r--r--. 1 root root 549K Nov 18 09:16 10.131.3.2.pcap
MASS
total 552K
-rw-r--r--. 1 root root 545K Nov 18 09:16 10.131.132.2.pcap
UTAH
total 548K
-rw-r--r--. 1 root root 543K Nov 18 09:16 10.132.5.2.pcap
NCSA
total 544K
-rw-r--r--. 1 root root 540K Nov 18 09:16 10.132.131.2.pcap
UCSD
total 4.0K
-rw-r--r--. 1 root root 24 Nov 18 07:35 10.134.130.2.pcap
FIU
total 4.0K
-rw-r--r--. 1 root root 24 Nov 18 07:36 10.135.133.2.pcap
CLEM
total 4.0K
-rw-r--r--. 1 root root 24 Nov 18 07:36 10.136.2.2.pcap
CERN
total 584K
-rw-r--r--. 1 root root 577K Nov 18 09:05 10.143.2.2.pcap


## Download the pcap files for analysis

In [29]:
for node in nodes:
    
    print(f"Downloading pcap files from {node.get_name()}")
    
    local_output_dir = '/home/fabric/work/my_notebooks/OWL_new/'
    owl.download_output(node, local_output_dir)

10.129.129.2.pcap
Downloaded /home/rocky/owl-output/10.129.129.2.pcap from STAR to /home/fabric/work/my_notebooks/OWL_new/STAR/10.129.129.2.pcap
10.130.7.2.pcap
Downloaded /home/rocky/owl-output/10.130.7.2.pcap from MAX to /home/fabric/work/my_notebooks/OWL_new/MAX/10.130.7.2.pcap
10.131.3.2.pcap
Downloaded /home/rocky/owl-output/10.131.3.2.pcap from MICH to /home/fabric/work/my_notebooks/OWL_new/MICH/10.131.3.2.pcap
10.131.132.2.pcap
Downloaded /home/rocky/owl-output/10.131.132.2.pcap from MASS to /home/fabric/work/my_notebooks/OWL_new/MASS/10.131.132.2.pcap
10.132.5.2.pcap
Downloaded /home/rocky/owl-output/10.132.5.2.pcap from UTAH to /home/fabric/work/my_notebooks/OWL_new/UTAH/10.132.5.2.pcap
10.132.131.2.pcap
Downloaded /home/rocky/owl-output/10.132.131.2.pcap from NCSA to /home/fabric/work/my_notebooks/OWL_new/NCSA/10.132.131.2.pcap
10.134.130.2.pcap
Downloaded /home/rocky/owl-output/10.134.130.2.pcap from UCSD to /home/fabric/work/my_notebooks/OWL_new/UCSD/10.134.130.2.pcap
10.13